In [1]:
import pandas as pd
import numpy as np 

# set_defog_dataset = pd.read_parquet('Data/Process/set_defog_data.parquet')
# test_defog_dataset = pd.read_parquet('Data/Process/test_defog_data.parquet')
# Test_defog_dataset = pd.read_csv('Data/test/defog/02ab235146.csv')

In [2]:
# def preprocessing_the_dataset(df):
#     def check_skewness(df):
#     # this can check relation between each column
#         skew_limit=0.75
#         skew_value=df[df.columns].skew()
#         #print(skew_value)
#         skew_col=skew_value[abs(skew_value)>skew_limit]
#         cols=skew_col.index
#         return cols

#     import random 
#     random_seed = 54
    
#     feature_col = ['AccV','AccML','AccAP']
#     label_col = ['StartHesitation','Turn','Walking', 'All_zero']

#     # make feature and label
#     feature_dataset = df[feature_col]
#     label_dataset = df[label_col]
    
#     # check skewness and powertransform
#     skew_columns = check_skewness(feature_dataset)
#     print(skew_columns)
    
#     print("Power Transform start")
#     from sklearn.preprocessing import PowerTransformer
#     pt=PowerTransformer(standardize=False)  
#     feature_dataset[skew_columns] = pt.fit_transform(feature_dataset[skew_columns])
    
#     print("Standardization start")
#     # Change features data to 0 and 1
#     from sklearn.preprocessing import StandardScaler
#     sc=StandardScaler()
#     feature_dataset=sc.fit_transform(feature_dataset)
    
#     print("Train test split begin")
#     from sklearn.model_selection import train_test_split
#     train_feature, valid_feature, train_label, valid_label = train_test_split(feature_dataset, label_dataset, test_size=0.2, random_state=True)
    
#     train_feature = np.array(train_feature) 
#     valid_feature = np.array(valid_feature)
#     train_label  = np.array(train_label)
#     valid_label = np.array(valid_label)
    
#     return train_feature, valid_feature, train_label, valid_label
    

In [3]:
# train_feature, valid_feature, train_label, valid_label = preprocessing_the_dataset(set_defog_dataset)

In [4]:
# train_feature.shape, valid_feature.shape, train_label.shape, valid_label.shape

In [5]:
# np.save('Data/defog/train_feature',train_feature)
# np.save('Data/defog/valid_feature',valid_feature)
# np.save('Data/defog/train_label',train_label)
# np.save('Data/defog/valid_label',valid_label)

#### bz of low memory, I save the features and label then load it

In [6]:
import pandas as pd
import numpy as np 

train_feature = np.load('Data/defog/train_feature.npy')
valid_feature = np.load('Data/defog/valid_feature.npy')
train_label = np.load('Data/defog/train_label.npy')
valid_label = np.load('Data/defog/valid_label.npy')

In [7]:
train_feature.shape, valid_feature.shape, train_label.shape, valid_label.shape

((29126467, 3), (7281617, 3), (29126467, 4), (7281617, 4))

### Build Model

In [8]:
# SKlearn model
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import  RadiusNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from catboost import Pool, CatBoostClassifier, cv

# import lightgbm as lgb


In [9]:


#Building Model Dict
Models = {                         
    "Decision Tree": DecisionTreeClassifier(),      
    # "KNearest": KNeighborsClassifier(n_jobs=-1),           
    # "Ridge" : RidgeClassifier(),              #poor result    
    # "MLP" : MLPClassifier(),                  #poor result              
    # "R_Neighour" : RadiusNeighborsClassifier(),
    # "Extra_T" : ExtraTreesClassifier(),
    # "R_forest" : RandomForestClassifier(),
    # "XGB" : xgb.XGBClassifier(),
    # "Catboost" : CatBoostClassifier()
    }



In [10]:
# # LGB

# lgb_train = lgb.Dataset(train_nm_features, train_nm_labels,  params={'verbose': -1})
# lgb_test = lgb.Dataset(train_nm_features, train_nm_labels,  params={'verbose': -1}, reference=lgb_train)

# LGBM = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_test], verbose_eval=False)
# LGBM_pred=LGBM.predict(test_nm_features)
# LGBM_pred = LGBM_pred.round()

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report

In [12]:
import numpy as np
np.random.seed(40)
from mlflow import MlflowClient
from mlflow.entities import ViewType
from mlflow.models.signature import infer_signature

# !mlflow server --backend-store-uri sqlite:///backend.db

In [13]:
def eval_metrics(classifier, test_features, test_labels):
    
    # make prediction
    predictions   = classifier.predict(test_features)
    
    base_score   = classifier.score(test_features,test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    av_precision = average_precision_score(test_labels, predictions)
    # Matrix = confusion_matrix(test_labels, predictions)
    
    # matrix_scores = { 
    #     "true negative"  : Matrix[0][0],
    #     "false positive" : Matrix[0][1],
    #     "false negative" : Matrix[1][0],
    #     "true positive " : Matrix[1][1]
    # }
    
    target_names = ['StartHesitation','Turn','Walking', 'All_zero']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions,target_names=target_names),"\n")
    # print("Confusion Matrix")
    # print("---------------------","\n")
    # print(f"{Matrix} \n")

    print("Accuracy Measures")
    print("---------------------","\n")
    print("Base score: ", base_score)
    print("Accuracy: ", accuracy)
    print("Avarge Precision: ", av_precision)
    
    return base_score,accuracy,av_precision

In [14]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


In [16]:
from sklearn.model_selection import cross_val_score
counter = 1
for Model_Name, classifier in Models.items(): 
    # with mlflow.start_run(nested=True):
    print(f"{counter}. {Model_Name}")
    
    with mlflow.start_run():
        # fit the model
        from joblib import parallel_backend

        with parallel_backend('threading', n_jobs=-1):
            classifier.fit(train_feature, train_label)
        cross_val_scores = cross_val_score(classifier, train_feature, train_label, cv=3)
        round_scores = round(cross_val_scores.mean(), 4) * 100
        print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round_scores , "% accuracy score")

        
        counter = counter + 1
        
        # Calculate the metrics
        base_score,accuracy,av_precision = eval_metrics(classifier,
                                                        valid_feature,
                                                        valid_label)  
        
        mlflow.log_param("Model"           , Model_Name)
        mlflow.log_metric("base_score"     , base_score)
        mlflow.log_metric("accuracy"       , accuracy)
        mlflow.log_metric("av_precision"   , av_precision)
        mlflow.log_metric("round_score", round_scores)
        
        signature = infer_signature(valid_feature, classifier.predict(valid_feature))
        if av_precision > 0.95 :
            mlflow.sklearn.log_model(classifier,Model_Name, signature=signature)
            print(f"f1 socre is more than 0.945 so the {Model_Name} is saved")
        else :
            print(f"Because f1 socre is not quality. The model is skip to saving phase.")
        
        print("________________________________________")

1. Decision Tree
Classifiers:  DecisionTreeClassifier Has a training score of 89.4 % accuracy score
Classification report
--------------------- 



/home/hanlinn/miniconda3/envs/main/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

StartHesitation       1.00      1.00      1.00   1822100
           Turn       0.86      0.88      0.87   1819501
        Walking       0.92      0.94      0.93   1819885
       All_zero       0.86      0.81      0.83   1820131

      micro avg       0.91      0.91      0.91   7281617
      macro avg       0.91      0.91      0.91   7281617
   weighted avg       0.91      0.91      0.91   7281617
    samples avg       0.91      0.91      0.91   7281617
 

Accuracy Measures
--------------------- 

Base score:  0.9089180329039552
Accuracy:  0.9089180329039552
Avarge Precision:  0.8522690981167873
Because f1 socre is not quality. The model is skip to saving phase.
________________________________________


In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
params = {
    "Decision Tree": {"criterion": ["gini", "entropy"], "max_depth": list(range(2,4,1)), 
              "min_samples_leaf": list(range(5,7,1))},
}

In [19]:
#Using Grid search for finding the most optimal hyperparameters
def gridsearch(classifier, params):
    grid_classifier = GridSearchCV(classifier, params)
    grid_classifier.fit(train_feature, train_label)
    best_classifier = grid_classifier.best_estimator_
    return best_classifier

In [24]:
print('Cross-Validation Scores after applying GridSearch:-')
for key, classifier in Models.items():
    best_classifier = gridsearch(classifier,params[key])
    cv_score = cross_val_score(classifier, train_feature, train_label, cv=3)
    print('{}: {}'.format(key,round(cv_score.mean()*100.0, 4)))

Cross-Validation Scores after applying GridSearch:-
Decision Tree: 89.396


In [22]:
best_classifier

DecisionTreeClassifier(max_depth=3, min_samples_leaf=5)

In [28]:
test_feature = np.load('Data/defog/test_feature.npy')
test_label = np.load('Data/defog/test_label.npy')

In [29]:
base_score,accuracy,av_precision = eval_metrics(best_classifier,
                                                test_feature,
                                                test_label)  

Classification report
--------------------- 



/home/hanlinn/miniconda3/envs/main/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hanlinn/miniconda3/envs/main/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

StartHesitation       0.58      0.44      0.50   2276249
           Turn       0.54      0.37      0.44   2273938
        Walking       0.00      0.00      0.00   2274579
       All_zero       0.78      0.29      0.42   2277255

      micro avg       0.61      0.27      0.38   9102021
      macro avg       0.48      0.27      0.34   9102021
   weighted avg       0.48      0.27      0.34   9102021
    samples avg       0.27      0.27      0.27   9102021
 

Accuracy Measures
--------------------- 

Base score:  0.27320734592899754
Accuracy:  0.27320734592899754
Avarge Precision:  0.3509588775975744


In [32]:
for Model_Name, classifier in Models.items(): 
    print(classifier)
    base_score,accuracy,av_precision = eval_metrics(classifier=classifier,
                                                    test_features=test_feature,
                                                    test_labels= test_label)  
    

DecisionTreeClassifier()
Classification report
--------------------- 

                 precision    recall  f1-score   support

StartHesitation       1.00      0.93      0.96   2276249
           Turn       0.81      0.86      0.83   2273938
        Walking       0.89      0.93      0.91   2274579
       All_zero       0.80      0.77      0.78   2277255

      micro avg       0.87      0.87      0.87   9102021
      macro avg       0.87      0.87      0.87   9102021
   weighted avg       0.87      0.87      0.87   9102021
    samples avg       0.87      0.87      0.87   9102021
 

Accuracy Measures
--------------------- 

Base score:  0.8708291268499601
Accuracy:  0.8708291268499601
Avarge Precision:  0.7969772649830308


/home/hanlinn/miniconda3/envs/main/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
